In [38]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal, interpolate
import copy
import os 
import tqdm
import pandas as pd
os.chdir('/home/kkotzen/research/PPG_sleepstaging_orion3/')
from pathlib import Path
from src.parsing.MESAParser import MESAParser


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
dl = MESAParser()
patients = ['6632'] #dl.database_all_patient_IDs

ppg = dl.load_signal(patients[0], 'Pleth')
ecg = dl.load_signal(patients[0], 'EKG')

ecg_peaks = dl.load_annotation(patients[0], 'ECG', 'epltd0', 'Peaks').astype(int)
valid_ecg_peaks = dl.load_annotation(patients[0], 'ECG', 'rpoint', 'Peaks').astype(int)
ecg_peaks = ecg_peaks[ecg_peaks < valid_ecg_peaks[-1]]
ecg_peaks = ecg_peaks
ecg_quality = dl.load_quality(patients[0], 'EKG')

ppg_peaks = dl.load_annotation(patients[0], 'Pleth', 'Aboy', 'Peaks')

#Limit them to the reference ECG
ppg_peaks = ppg_peaks[ppg_peaks<ecg_peaks[-1]]

sleep = dl.load_sleep(patients[0])
sleep[sleep==4]=3

In [ ]:
def duplicate(x):
    return np.array([[x_]*2 for x_ in x]).flatten()

def hold_line(t, x):
    t = [i for i in t]
    x = [i for i in x]
    
    if len(t) - len(x) == 0:
            t.append(t[-1]+t[-1]-t[-2])
    if len(t)-len(x) != 1:
            raise ValueError()
            
    t_ = duplicate(t)[1:-1]
    x_ = duplicate(x)
    return t_, x_


def plot_hypnogram(sleep_stages, ax1, ticks={0: 'W', 1: 'L', 2:'D', 3:'R'}):

    t_, label_ = hold_line(np.arange(sleep_stages.shape[0])*30, sleep_stages)
    ax1.plot(t_, label_, color='black', label='Labeled sleep stage')
    ax1.set_ylabel('Sleep Stage', fontsize=10)
    ax1.set_yticks(list(ticks.keys()))
    ax1.set_yticklabels(list(ticks.values()), rotation='vertical')
    

def plot_hypnogram_seconds(sleep_stages, ax1, ticks={0: 'W', 1: 'L', 2:'D', 3:'R'}):

    t_, label_ = hold_line(np.arange(0, sleep_stages.shape[0]/(60*60)*30, 30/(60*60)), sleep_stages)
    ax1.plot(t_, label_, color='black', label='Labeled sleep stage')
    ax1.set_ylabel('Sleep Stage', fontsize=10)
    ax1.set_xlabel('Time (Hour)', fontsize=10)

    ax1.set_yticks(list(ticks.keys()))
    ax1.set_yticklabels(list(ticks.values()), rotation='vertical')
    plt.tight_layout()

In [ ]:
dl = SpanishParser()
patient = dl.database_all_patient_IDs[5]
sleep = dl.load_sleep(patient)[0:7*2*60]
sleep[sleep > 3] =0
plt.close("all")
plt.figure(figsize=(5.93, 1.6))
ax1=plt.subplot(1,1,1)
plot_hypnogram_seconds(sleep, ax1)
ax1.spines['right'].set_visible(False)
ax1.spines['top'].set_visible(False)
plt.savefig("./notebooks/Images/example_normal_hypnogram.png")

In [ ]:
dl = MESAParser()
patient = '0527'# dl.database_all_patient_IDs[i+40]
sleep = dl.load_sleep(patient) [0:7*2*60]
sleep[sleep==2]=1
sleep[sleep==3]=2
sleep[sleep==4]=3
sleep[sleep > 3] =0
plt.close("all")
plt.figure(figsize=(5.93, 1.6))
ax1=plt.subplot(1,1,1)
plot_hypnogram_seconds(sleep, ax1)
ax1.spines['right'].set_visible(False)
ax1.spines['top'].set_visible(False)
plt.savefig(f"./notebooks/Images/example_irregular_hypnogram.png")

In [ ]:



t_ppg = np.arange(0, len(ppg)/256, 1/256)

plt.close("all")
cm = 1/2.54  # centimeters in inches
plt.figure(figsize=(7.9*cm, 5*cm))
ax1=plt.subplot(2,1,1)
plot_hypnogram(sleep, ax1)
ax1.spines['right'].set_visible(False)
ax1.spines['top'].set_visible(False)

ax2=plt.subplot(2,1,2, sharex=ax1)
plt.plot(t_ppg, ppg, color='black')
plt.xlabel("Time (Seconds)", fontsize=10)
plt.ylabel('PPG (mV)', fontsize=10)
ax2.spines['right'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax2.axes.xaxis.set_ticklabels([])
ax2.axes.yaxis.set_ticklabels([])

plt.xticks(fontsize= 8)
plt.yticks(fontsize= 8)
plt.tight_layout()

In [ ]:
plt.close("all")
cm = 1/2.54  # centimeters in inches
plt.figure(figsize=(7.9*cm, 5*cm))
ax1=plt.subplot(2,1,1)
plt.plot(ecg)
plt.ylabel('ECG (mV)', fontsize=8)

ax1.spines['right'].set_visible(False)
ax1.spines['top'].set_visible(False)
ax1.axes.xaxis.set_ticklabels([])
ax1.axes.yaxis.set_ticklabels([])

ax2=plt.subplot(2,1,2, sharex=ax1)
plt.plot(ppg)
plt.xlabel("Time (Seconds)", fontsize=8)
plt.ylabel('PPG (mV)', fontsize=8)

ax2.spines['right'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax2.axes.xaxis.set_ticklabels([])
ax2.axes.yaxis.set_ticklabels([])

plt.xticks(fontsize= 8)
plt.yticks(fontsize= 8)
plt.tight_layout()

In [ ]:
plt.savefig("./notebooks/Images/PTT_from_ecg.svg")

In [ ]:
from matplotlib import gridspec

plt.close('all')
t = np.arange(0, len(ecg)/256, 1/256)

fig = plt.figure(figsize=(8, 6)) 
gs = gridspec.GridSpec(2, 1, height_ratios=[1,3]) 
ax0 = plt.subplot(gs[1])
ax0.plot(t, ecg)
ax0.plot(valid_ecg_peaks/256, 2*np.ones_like(valid_ecg_peaks))
ax0.plot(valid_ecg_peaks/256, ecg[valid_ecg_peaks], 'o')
ax0.plot(valid_ecg_peaks/256, 2*np.ones_like(valid_ecg_peaks), '>')
ax0.set_xlabel('Time (seconds)')
ax0.set_ylabel('Amplitude')
# ax0.set_yticks([])
# ax0.set_xticks([])

ax1 = plt.subplot(gs[0], sharex=ax0)
ax1.plot(np.gradient(valid_ecg_peaks)/256)
ax1.set_ylabel('R-R Interval (seconds)')
# ax1.set_yticks([])

In [ ]:
start, stop = 256*2800, 256*3100

ecg_peaks_ = valid_ecg_peaks[(valid_ecg_peaks > start)*(valid_ecg_peaks < stop)]
ppg_peaks_ = ppg_peaks[(ppg_peaks > start)*(ppg_peaks < stop)]

IHR = (np.gradient(ecg_peaks_)/256)
IPR =  (np.gradient(ppg_peaks_)/256)

d = np.arange(start+256*5, stop-256*5, 128)
t = np.arange(0, len(d)/2, 0.5)
IHR_cont = interpolate.interp1d(ecg_peaks_, IHR)
IPR_cont = interpolate.interp1d(ppg_peaks_-120, IPR)

IHR = IHR_cont(d)
IPR = IPR_cont(d)

plt.close("all")
fig, ax = plt.subplots()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.plot(t,IHR, label='HR (ECG derived)')
ax.plot(t,IPR, label='PR (PPG derived)', alpha=0.7)
ax.set_xlabel("Time (seconds)")
ax.set_ylabel('Interval (seconds)')
ax.legend(loc='upper right')

fig, ax = plt.subplots()
ax.psd(IHR, NFFT=128, Fs=2, label="HR (ECG derived)")
ax.psd(IPR, NFFT=128, Fs=2, label="PR (PPG derived)")
ax.grid(False)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.legend(loc='upper right')

In [ ]:
import numpy as np
def genSine(f0, fs, dur):
    t = np.arange(dur)
    sinusoid = np.sin(2*np.pi*t*(f0/fs))
    return sinusoid

dl = MESAParser()
patients = ['0002'] #dl.database_all_patient_IDs

ppg = dl.load_signal(patients[0], 'Pleth')


fifty_hz = genSine(50, 256, len(ppg))
ppg_noisy = ppg+0.0125*fifty_hz
plt.close("all")
plt.figure(figsize=(2.9,1.6))
plt.plot(ppg_noisy, label='Noisy')
plt.plot(ppg, label='Filtered')
plt.ylabel('PPG (N/A)', fontsize=10)
plt.xlabel("Time (Seconds)",fontsize=10)
plt.legend(loc='upper right', fontsize=6)
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.axes.xaxis.set_ticklabels([])
ax.axes.yaxis.set_ticklabels([])
plt.tight_layout()


In [ ]:
plt.savefig("./notebooks/Images/powerline.png")

In [ ]:
dl = MESAParser()
patients = ['0002'] #dl.database_all_patient_IDs

ppg = dl.load_signal(patients[0], 'Pleth')

plt.close("all")
plt.figure(figsize=(2.9,1.6))
plt.plot(ppg)
plt.ylabel('PPG (N/A)', fontsize=10)
plt.xlabel("Time (Seconds)",fontsize=10)
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.axes.xaxis.set_ticklabels([])
ax.axes.yaxis.set_ticklabels([])
plt.tight_layout()

In [ ]:
plt.savefig("./notebooks/Images/motion.png")

In [ ]:
import numpy as np
def genSine(f0, fs, dur):
    t = np.arange(dur)
    sinusoid = np.sin(2*np.pi*t*(f0/fs))
    return sinusoid

dl = MESAParser()
patients = ['0002'] #dl.database_all_patient_IDs
ppg = dl.load_signal(patients[0], 'ECG')
peaks = dl.load_annotation(patients[0], 'Pleth', 'Aboy', 'Peaks').astype(int)
ecg_peaks = dl.load_annotation(patients[0], 'EKG', 'epltd0', 'Peaks').astype(int)
valid_ecg_peaks = dl.load_annotation(patients[0], 'EKG', 'rpoint', 'Peaks').astype(int)
ecg_peaks = ecg_peaks[ecg_peaks < valid_ecg_peaks[-1]]
ecg_peaks = ecg_peaks

plt.close("all")
plt.figure(figsize=(2.9,1.6))
plt.plot(ppg, label='Signal Artifacts')
plt.plot(peaks, ppg[peaks], 'x')
plt.plot(ecg_peaks+100, ppg[ecg_peaks+100], '*')
plt.ylabel('PPG Amplitude(N/A)')
plt.xlabel("Time (Seconds)")
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.axes.xaxis.set_ticklabels([])
ax.axes.yaxis.set_ticklabels([])
plt.tight_layout()

In [79]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import os 
os.chdir('/home/kkotzen/research/PPG_sleepstaging/')
from pathlib import Path
from src.parsing.MESAParser import MESAParser

from scipy import signal, stats

def butter_bandpass(lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    sos = signal.butter(order, [low, high], btype='band', output='sos')
    return sos

def butter_bandpass_filter(data, lowcut, highcut, fs, order=4):
    sos = butter_bandpass(lowcut, highcut, fs, order=order)
    y = signal.sosfiltfilt(sos, data)
    return y

def standardize_signal(data):
    return stats.zscore(data.astype(np.float32)).astype(np.float16)

fs = 256
highcut = 8
lowcut = 0.5
order=8

plt.close("all")
plt.figure(1, figsize=(2.9,1.6))
plt.clf()
order_words = {2: "2nd", 4: "4th", 8: "8th"}

order=8
sos = butter_bandpass(lowcut, highcut, fs, order=order)
w, h = signal.sosfreqz(sos, worN=128)
plt.plot((fs * 0.5 / np.pi) * w, 20 * np.log10(abs(h)+0.000001), label=f"Butter {order_words[order]} order")
    
plt.xlabel('Frequency (Hz)', fontsize=10)
plt.ylabel('Gain (dB)', fontsize=10)
plt.xlim((0,12))
plt.ylim((-40, 2.5))
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.tight_layout()
plt.savefig("/home/kkotzen/images/thesis/bandpass_filter_frequency.png")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [66]:
ppg_filt = signal.sosfiltfilt(sos, ppg)

plt.close('all')
plt.figure(figsize=(2.9,1.6))
plt.psd(10*ppg, NFFT=2056, Fs=256, label='Raw')
plt.psd(10*ppg_filt, NFFT=2056, Fs=256, label='Filtered')
plt.xlim([0,15])
plt.ylim([-50,10])
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.ylabel("PSD(dB/Hz)")
plt.xlabel("Frequency(Hz)")
plt.tight_layout()
plt.grid(False)
plt.legend(fontsize=6)
plt.savefig("/home/kkotzen/images/thesis/bandpass_psd.png")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [63]:
plt.close('all')
plt.figure(figsize=(5.9,1.8))
x = 120
plt.plot(ppg[256*60*x:int(256*60*(x+0.125))], label='Raw')
plt.plot(ppg_filt[256*60*x:int(256*60*(x+0.125))], label='Filtered')
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.axes.xaxis.set_ticklabels([0,1,2,3,4,5,6,7,8,9])
ax.axes.yaxis.set_ticklabels([])
plt.ylabel("PPG (N/A)", fontsize=10)
plt.xlabel("Time (seconds)", fontsize=10)
plt.tight_layout()
plt.grid(False)
leg = plt.legend(loc='upper right', fontsize=6)
leg.get_frame().set_linewidth(0.0)
plt.savefig("/home/kkotzen/images/thesis/bandpass_filtered_unfiltered.png")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [108]:
def cheby_lowpass_sos(highcut, fs, order=8):
    nyq = 0.5 * fs
    high = highcut / nyq
    sos = signal.cheby2(order, 40, high, btype='lowpass', output='sos')
    return sos


fs = 256
highcut = 8
order=8

plt.close("all")
plt.figure(1, figsize=(2.9,1.6))
plt.clf()

order=8
sos = cheby_lowpass_sos(highcut, fs, order=order)
w, h = signal.sosfreqz(sos, worN=128)
plt.plot((fs * 0.5 / np.pi) * w, 20 * np.log10(abs(h)+0.000001))
    
plt.xlabel('Frequency (Hz)', fontsize=10)
plt.ylabel('Gain (dB)', fontsize=10)
plt.xlim((0,12))
plt.ylim((-40, 2.5))
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.tight_layout()
plt.savefig("/home/kkotzen/images/thesis/lowcut_filter_frequency.png")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [107]:
ppg_filt = signal.sosfiltfilt(sos, ppg)

plt.close('all')
plt.figure(figsize=(2.9,1.6))
plt.psd(10*ppg, NFFT=2056, Fs=256, label='Raw')
plt.psd(10*ppg_filt, NFFT=2056, Fs=256, label='Filtered')
plt.xlim([0,15])
plt.ylim([-50,10])
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.ylabel("PSD(dB/Hz)")
plt.xlabel("Frequency(Hz)")
plt.tight_layout()
plt.grid(False)
plt.legend(fontsize=6)
plt.savefig("/home/kkotzen/images/thesis/lowpass_psd.png")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [106]:
plt.close('all')
plt.figure(figsize=(5.9,1.8))
x = 120
z = 0.125
plt.plot(ppg[256*60*x:int(256*60*(x+z))], label='Raw', linewidth=2)
plt.plot(ppg_filt[256*60*x:int(256*60*(x+z))], label='Filtered', alpha=1)
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.axes.xaxis.set_ticklabels([0,1,2,3,4,5,6,7,8,9])
ax.axes.yaxis.set_ticklabels([])
plt.ylabel("PPG (N/A)", fontsize=10)
plt.xlabel("Time (seconds)", fontsize=10)
plt.tight_layout()
plt.grid(False)
leg = plt.legend(loc='upper right', fontsize=6)
leg.get_frame().set_linewidth(0.0)
plt.savefig("/home/kkotzen/images/thesis/lowpass_filtered_unfiltered.png")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:
plt.close('all')
plt.figure(figsize=(5.9,2.5))
plt.plot(np.gradient(ppg[256*60*60:256*60*90]), label='Raw')
plt.plot(np.gradient(ppg_filt[256*60*60:256*60*90]), label='Filtered')
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.axes.xaxis.set_ticklabels([])
ax.axes.yaxis.set_ticklabels([])
plt.ylabel("dPPG Amplitude (N/A)")
plt.xlabel("Time (seconds)")
plt.tight_layout()
plt.grid(False)
leg = plt.legend(loc='upper right')
leg.get_frame().set_linewidth(0.0)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
import numpy as np
def genSine(f0, fs, dur):
    t = np.arange(dur)
    sinusoid = np.sin(2*np.pi*t*(f0/fs))
    return sinusoid

dl = MESAParser()
patients = ['0002'] #dl.database_all_patient_IDs
ppg = dl.load_signal(patients[0], 'Pleth')
peaks = dl.load_annotation(patients[0], 'Pleth', 'Aboy', 'Peaks')
onsets = dl.load_annotation(patients[0], 'Pleth', 'Aboy', 'Onsets')
plt.close("all")
plt.figure(figsize=(4,2.5))
plt.plot(ppg)
plt.plot(peaks, ppg[peaks], 'x')
plt.plot(onsets, ppg[onsets], 'o')
plt.ylabel('PPG Amplitude(N/A)')
plt.xlabel("Time (Seconds)")
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.axes.xaxis.set_ticklabels([])
ax.axes.yaxis.set_ticklabels([])
plt.tight_layout()

**EEG Diagram**

In [ ]:
from src.parsing.SpanishParser import SpanishParser

from src.parsing.utils.extract_numpy_from_edf import extract_numpy_from_edf
dl = SpanishParser()
patients = dl.database_all_patient_IDs
patient = '377'
eeg = extract_numpy_from_edf(dl.path_edf(patient), 'Cz')

In [ ]:
plt.close('all')
sleep = dl.load_sleep(patient)
t_sleep = np.arange(0, len(sleep)*30, 30)
t_eeg = np.arange(0, len(eeg)/512, 1/512)
ax1 = plt.subplot(2,1,1)
plt.plot(t_sleep[sleep==0], sleep[sleep==0])
ax2 = plt.subplot(2,1,2, sharex=ax1)
plt.plot(t_eeg, eeg)

# Compare sleep stage prevelance visually

In [11]:
dl = MESAParser()
patients = dl.database_all_patient_IDs

df = pd.DataFrame()

for patient in tqdm.tqdm(patients):
    sleep = dl.load_sleep(patient)
    sleep[sleep==2]=1
    sleep[sleep==3]=2
    sleep[sleep==4]=3
    sleep[sleep > 3] =0
    stage, count = np.unique(sleep, return_counts=True)
    df = df.append({s:c for s, c in zip(stage,count)}, ignore_index=True)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2054/2054 [00:18<00:00, 113.67it/s]


In [20]:
df.sum()/df.sum().sum()

0.0    0.433062
1.0    0.406419
2.0    0.057375
3.0    0.103144
dtype: float64

In [31]:
plt.close('all')
fig, ax = plt.subplots(1,1, figsize=(4,4))
ax.bar([1,2,3,4], df.sum()/df.sum().sum(), color=['red', 'green', 'blue', 'cyan'])
ax.set_xticks([1,2,3,4])
ax.set_xticklabels(["Wake", "Light", "Deep", "REM"])
ax.set_xlabel("Sleep Stage")
ax.set_ylabel("Prevelance (%)")
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
x = {0:0.6084310206828908, 1: 0.5911641609954276, 2: 5.030164819659377, 3: 2.3458207598308074}.values()
x = np.array(list(x))

In [37]:
x/min(x)

array([1.02920823, 1.        , 8.50891369, 3.96813764])

In [63]:
dl = MESAParser()
patients = ['3180'] #dl.database_all_patient_IDs

ppg = dl.load_signal(patients[0], 'ECG')
qppg = dl.load_annotation(patients[0], 'ECG', 'epltd0', 'Peaks')

print(qppg)
plt.close('all')
plt.plot(ppg)
# plt.plot(qppg, ppg[qppg], 'x')

[]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [58]:
fs = 256
beats = qppg
resample_fs = 2

beats = beats / fs
ibi = np.diff(beats)
beats = beats[:-1] + (beats[1:] - beats[:-1]) / 2
upper = np.percentile(ibi, 99)
lower = np.percentile(ibi, 1)
ibi_filt_idx = np.where((ibi < upper) & (ibi > lower))
ibi = ibi[ibi_filt_idx]
beats = beats[ibi_filt_idx]
ihr = 1 / ibi
x_2hz = np.arange(0, beats[-1], 1 / resample_fs)
ihr_2hz = np.interp(x_2hz, beats, ihr)
ihr_2hz = (ihr_2hz - np.mean(ihr_2hz)) / np.std(ihr_2hz)


In [59]:
plt.close('all')
plt.plot(ihr_2hz)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [53]:
np.percentile(ibi, 99)

1.09375